In [ ]:
!ls path

In [ ]:
import dataset
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
from PIL import Image
from matplotlib import pyplot as plt
import albumentations
%matplotlib inline
path = "/home/gianscarpe/dev/data/dhp19/time_count_dataset/movements_per_frame/"

In [ ]:
from albumentations import *
from albumentations.pytorch import ToTensor

In [ ]:
# Indexes
from utils.generate_indexes import save_npy_indexes_and_map
file_paths, train_index, val_index, test_index = save_npy_indexes_and_map(path, split_at=0.8)

In [ ]:
data_index = np.concatenate([train_index, val_index])
print(data_index.shape)

In [ ]:
transform_val = Compose([
    CenterCrop(224, 224),
    Normalize(mean=0.0074, std=0.0553),
    ToTensor()
])

In [ ]:
d = dataset.get_dataset(file_paths, index=val_index, preload=False, n_channels=1, preprocess=transform_val)

In [ ]:
len(d.x_indexes)

In [ ]:
#from https://discuss.pytorch.org/t/computing-the-mean-and-std-of-dataset/34949/2
loader = DataLoader(
    d,
    batch_size=32,
    num_workers=1,
    shuffle=False
)

## Mean and Std

In [ ]:
mean = 0.
std = 0.
nb_samples = 0.
counts = torch.zeros(33)
for data, labels in loader:
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples
    for i in labels:
        counts[i]+=1

mean /= nb_samples
std /= nb_samples

In [ ]:
print(f"Mean: {mean} | Std: {std}")

In [ ]:
counts = torch.zeros(33)
for data, i in d:
    counts[i]+=1

plt.bar(np.arange(1, 34), counts.numpy().astype('int'))

## Visually evaluate loader

In [ ]:
from torchvision import transforms

In [ ]:
it = iter(loader)
b_x, b_y = next(it)
trans = transforms.ToPILImage()
for img, label in zip(b_x, b_y):
    img = trans(img)
    plt.figure()
    plt.imshow(img)

In [ ]:
b_x.std()